In [2]:
import numpy as np
import pandas as pd
from   scipy.stats import norm
import scipy.optimize as opt
import math

In [3]:
df = pd.read_stata('data/mturk_clean_data_short.dta')

In [4]:
df

,buttonpresses,treatment,treatmentname
0,45,1.1,1c PieceRate
1,100,1.1,1c PieceRate
2,114,1.1,1c PieceRate
3,115,1.1,1c PieceRate
4,126,1.1,1c PieceRate
...,...,...,...
9856,3031,10,Gift Exchange
9857,3060,10,Gift Exchange
9858,3073,10,Gift Exchange
9859,3135,10,Gift Exchange


In [5]:
genre = ['payoff_per_100','payoff_charity_per_100',
         'dummy_charity','delay_wks','delay_dummy','gift_dummy','prob','weight_dummy'] 
for i in genre:
    if i != 'prob':
        df[F"{i}"] = 0
    else:
        df[F"{i}"] = 1

In [6]:
payoff_list = ['1.1','1.2','1.3','2','1.4','4.1','4.2','6.2','6.1']
payoff = [0.01,0.1,0.0,0.001,0.04,0.01,0.01,0.02,1]
charity_list = ['3.1','3.2']
charity_payoff = [0.01,0.1]
delay_list = ['4.1','4.2']
delay_wks = [2,4]
prob_list = ['6.2','6.1']
probability = [0.5,0.01]

In [7]:
assum = {'payoff_per_100':{'1.1':0.01,'1.2':0.1,'1.3':0.0,'2':0.001,'1.4':0.04,'4.1':0.01,'4.2':0.01,'6.2':0.02,'6.1':1},
 'payoff_charity_per_100':{'3.1':0.01,'3.2':0.1},
 'dummy_charity':{'3.1':1,'3.2':1},
 'delay_wks':{'4.1':2,'4.2':4},
 'delay_dummy':{'4.1':1,'4.2':1},
 'prob':{'6.2':0.5,'6.1':0.01},
 'weight_dummy':{'6.1':1},
 'gift_dummy':{'10':1}
 }

In [8]:
for task,payoff in assum.items():
    for key in payoff:
        df.loc[df.treatment == key, task] = payoff[key]

In [9]:
df['buttonpresses'] += 0.1 # python rounds 50 to 0, while stata to 100. by adding a small value we avoid this mismatch
df['buttonpresses_nearest_100'] = round(df['buttonpresses'],-2)
df.loc[df.buttonpresses_nearest_100 == 0, 'buttonpresses_nearest_100'] = 25
df['logbuttonpresses_nearest_100']  = np.log(df['buttonpresses_nearest_100'])

In [10]:
# Estimate procedure for s, k, gamma in benchmark case with exp cost function
# Define the benchmark sample by creating dummies equal to one if in treatment 1.1, 1.2, 1.3 

df['dummy1'] = (df['treatment'].isin(['1.1', '1.2','1.3'])).astype(int)# isin can be applied in pandas dataframe

In [715]:
# Set the initial values for the optimization procedure and scalers for k and s in the exp cost function case

st_values_exp = [0.015645717, 1.69443, 3.69198]#gamma_init_exp, k_init_exp, s_init_exp
st_values_power = [19.8117987, 1.66306e-10, 7.74996]#gamma_init_power, k_init_power, s_init_power
bp52_aut = [20.546,5.12e-70,3.17e-06]


In [716]:
def opt_param(effort, k_scaler, s_scaler, st_values,type):
    def benchmark(pay100, g, k, s):
        if type == "exp":
            check1 = k/k_scaler            # 'first'  component to compute f(x,θ). We call it check1 since it will enter a log, so we need to be careful with its value being > 0
            check2 = s/s_scaler + pay100
        else:
            check1= max(k/k_scaler, 1e-115)                  # since check1 will enter log it must be greater than zero
            check2= np.maximum(s/s_scaler + pay100, 1e-10)   # np.maximum computes the max element wise. We do not want a negative value inside log    
           # 'second' component to compute f(x,θ)   
        f_x = (-1/g * np.log(check1) +1/g * np.log(check2)) # f(x,θ) written above
        return f_x
    sol = opt.curve_fit(benchmark,
                    df.loc[df['dummy1']==1].payoff_per_100,
                    df.loc[df['dummy1']==1,effort],maxfev = 5000,
                    p0 = st_values)
    
    se = np.sqrt(np.diagonal(sol[1]))
    solo = [i/j for i,j in zip(sol[0],[1,k_scaler,s_scaler])]
    se = [i/j for i,j in zip(se,[1,k_scaler,s_scaler])]
    #Following code are for making comparable and understandable result
    sol_result = [0]*3
    se_result = [0]*3
    
    for i in range(0,len(sol[0])):
        if i == 0:
            sol_result[i] = round(solo[i],3)
        else:
            sol_result[i] = '{0:.2e}'.format(Decimal(solo[i]))
            se_result[i] = '{0:.2e}'.format(Decimal(se[i]))   
    
    if type == "exp":
        #return sol[0],np.sqrt(np.diagonal(sol[1]))
        return sol_result,se_result
    else:
        sse = np.round(np.sum((benchmark(df.loc[df['dummy1']==1].payoff_per_100,*sol[0])-df.loc[df['dummy1']==1,effort])**2),3)
        #bp52_aut = [20.546,5.12e-13,3.17]
        sse_aut = np.round(np.sum((benchmark(df.loc[df['dummy1']==1].payoff_per_100,*bp52_aut)-df.loc[df['dummy1']==1,effort])**2),3)
        #return sol[0],np.sqrt(np.diagonal(sol[1])),sse,sse_aut
        return sol_result,se_result,sse,sse_aut
    

In [717]:
be54,se54 = opt_param(effort = "buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,st_values = st_values_exp,type = "exp")
be54

[0.016, '1.71e-16', '3.72e-6']

In [718]:
aut = [0]*3
for i in range(0,len(bp52_aut)):
        if i == 0:
            aut[i] = round(bp52_aut[i],3)
        else:
            aut[i] = '{0:.2e}'.format(Decimal(bp52_aut[i]))
aut

[20.546, '5.12e-70', '3.17e-6']

In [227]:
#Re-write the scaler as list for calculate the estimation parameter
#scal = [1,1e+16,1e+6]
#p_scal = [1,1e+57,1e+6]
#para = [i/j for i,j in zip(be54[0] ,scal)]
#se_para = [i/j for i,j in zip(be54[1] ,scal)]

In [228]:
#para

[0.01564107084478763, 1.70926793259111e-16, 3.7222599348152786e-06]

In [719]:
bp52,sp52,sse,sse_aut = opt_param(effort = "logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,st_values = st_values_power,type = "power")
#p_para = [i/j for i,j in zip(bp52[0] ,p_scal)]
#se_p_para = [i/j for i,j in zip(bp52[1] ,p_scal)]

In [720]:
sse_aut

42176.046

In [16]:
#The previous function "opt_param" is the replication of the original code but in a more efficient way. 
#However, the result of the code is somehow not very close to the result of the paper. 
#So change part of the assumption of the code.I rename the func as my_ver_opt and try to get closer to the result of the paper and also makes the code simpler! 

In [17]:
def my_ver_opt(effort,st_values):
    def my_benchmark(pay100, g, k, s):
        if k <= 0 or (s+pay100).values.min() <= 0:#making sure that the value in log function would not be negative
            return 1e-16
        f_x = (-1/g * np.log(k) +1/g * np.log(s+pay100))   # f(x,θ) written above
    #f_x = np.clip(f_x, a_min=1e-16, a_max=None)
        return f_x
    sol = opt.curve_fit(my_benchmark,
                        df.loc[df['dummy1']==1].payoff_per_100,
                        df.loc[df['dummy1']==1,effort],
                        p0 = st_values,maxfev = 5000)
    return sol[0],np.sqrt(np.diagonal(sol[1]))

In [18]:
my_modified = my_ver_opt("buttonpresses_nearest_100",st_values=st_values_exp)# This result is closer to NLS on individual effort on Exponential effort cost.

In [19]:
my_modified_power = my_ver_opt("logbuttonpresses_nearest_100",st_values=st_values_power)

In [610]:
#The original python code only replicate the power cost of effort, but here I also calculate the estimation of Exponential cost of effort.
def other_opi(effort, k_scaler,s_scaler,type,opt_type,st_values):
    def benchmark_other(params):
        pay100 = np.array(df.loc[df['dummy1']==1].payoff_per_100)
        buttonpresses = np.array(df.loc[df['dummy1']==1,effort])
        g, k, s = params
        if type == "exp":
            check1 = k/k_scaler            # 'first'  component to compute f(x,θ). We call it check1 since it will enter a log, so we need to be careful with its value being > 0
            check2 = s/s_scaler + pay100
        elif type == "power":   
            check1= max(k/k_scaler, 1e-100)
            check2= np.maximum(s/s_scaler + pay100, 1e-10)   
        if opt_type == "ls":    
            f_x = (0.5*((-1/g * np.log(check1) +1/g * np.log(check2))-buttonpresses)**2)
        else:
            f_x = np.sum(0.5*((-1/g * np.log(check1) +1/g * np.log(check2))-buttonpresses)**2) 
        return f_x
    sol_result = [0]*3
    if opt_type == "ls":#least squared
        sol = opt.least_squares(benchmark_other,
                        x0 = st_values,
                        xtol=1e-15,
                        ftol=1e-15,
                        gtol=1e-15,
                        method='lm')
        sse = np.round((2*benchmark_other(sol.x)).sum(),3)
        solo = [i/j for i,j in zip(sol.x,[1,k_scaler,s_scaler])]
        for i in range(0,3):
            if i==0:
                sol_result[i] = round(solo[i],3)   
            else:
                sol_result[i] = '{0:.2e}'.format(Decimal(solo[i]))
        #soloo = np.round([*solo],4)
        #se = [i/j for i,j in zip(se,[1,k_scaler,s_scaler])]
        #return sol.x,sse
        return sol_result,sse
    
    elif opt_type == "mini":
        sol = opt.minimize(benchmark_other,
                       x0 = st_values,
                       method='Nelder-Mead',
                       options={'maxiter': 2500})
        solo = [i/j for i,j in zip(sol.x,[1,k_scaler,s_scaler])]
        #se = [i/j for i,j in zip(se,[1,k_scaler,s_scaler])]
        #sse = 2*benchmark_other(sol.x)
        sse = np.round((2*benchmark_other(sol.x)),3)
        for i in range(0,3):
            if i==0:
                sol_result[i] = round(solo[i],3)   
            else:
                sol_result[i] = '{0:.2e}'.format(Decimal(solo[i]))
        #return sol.x ,sse  
        return sol_result,sse 

In [631]:
bp52_least_squaree = other_opi("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type="power",opt_type="ls",st_values = st_values_power)
#bp52_least_square = [i/j for i,j in zip(bp52_least_squaree[0],p_scal)]
bp52_least_squaree

([21.787, '2.17e-69', '9.56e-8'], 1112.339)

In [611]:
bp52_optt = other_opi("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type = "power",opt_type="mini",st_values = st_values_power)
#bp52_opt = [i/j for i,j in zip(bp52_optt[0],p_scal)]
bp52_optt

([21.266, '3.45e-72', '1.33e-6'], 670.61)

In [233]:
scal = [1,1e+16,1e+6]
scal_power = [1,1e+57,1e+6]
be52_ls = other_opi("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",opt_type="ls",st_values=st_values_exp)
be52_least_square = [i/j for i,j in zip(be52_ls[0],scal)]
be52_least_square


[0.011208005206028106, 8.218133126797552e-12, 0.00011621467469798861]

In [242]:
be52_optt = other_opi("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",opt_type="mini",st_values=st_values_exp)
be52_opt = [i/j for i,j in zip(be52_optt[0],scal)]
be52_opt

[0.015641086579831008, 1.7092148653273357e-16, 3.7222415758983223e-06]

In [621]:
bp52

['2.12e+1', '5.95e-72', '1.38e-6']

In [703]:
from IPython.display import display
pn = ["Curvature γ of cost function","Level k of cost of effort", "Intrinsic motivation s","Min obj. function"]
bp52_aut = [20.546,5.12e-70,3.17e-06]
r1 = pd.DataFrame({'parameters':pn,'curve_fit':[*bp52,sse],
                   'least_square':[*bp52_least_squaree[0],bp52_least_squaree[1]],
                   'minimize_nd':[*bp52_optt[0],bp52_optt[1]],
                   'authors':[*aut,sse_aut]})

In [704]:
display(r1)

,parameters,curve_fit,least_square,minimize_nd,authors
0,Curvature γ of cost function,21.194,21.787,21.266,20.546
1,Level k of cost of effort,5.95e-72,2.17e-69,3.45e-72,5.12e-70
2,Intrinsic motivation s,1.38e-6,9.56e-8,1.33e-6,3.17e-6
3,Min obj. function,670.61,1112.339,670.61,42176.046


In [260]:
display(r1)# This is the outcome of power cost of effort from original code

,parameters,curve_fit,least_square,minimize_nd,authors
0,Curvature γ of cost function,2.119386e+01,2.178728e+01,2.126564e+01,2.054600e+01
1,Level k of cost of effort,5.946298e-72,2.170430e-69,3.453580e-72,5.120000e-70
2,Intrinsic motivation s,1.377482e-06,9.555272e-08,1.331358e-06,3.170000e-06
3,Min obj. function,6.706101e+02,1.112339e+03,6.706100e+02,6.723867e+02


In [90]:
#Here I also adding the outcome of exponential cost of effort
pn = ["Curvature γ of cost function","Level k of cost of effort", "Intrinsic motivation s"]
be52_aut = [0.0156,1.71E-16,3.72E-06]
r2 = pd.DataFrame({'parameters':pn,'curve_fit':[*para],
                   'least_square':[*be52_least_square],
                   'minimize_nd':[*be52_optt[0]],"my_modified_curve_fit":[*my_modified],
                   'authors':[*be52_aut]})

In [91]:
display(r2) 
#The estimation from the original code of parameter "k" and "s" seems a bit different from the authors. 
#I think it is result from the basic assumption of the model.
#I tried to modified it from curve fit. 

,parameters,curve_fit,least_square,minimize_nd,my_modified_curve_fit,authors
0,Curvature γ of cost function,1.564107e-02,1.120801e-02,1.564109e-02,1.564107e-02,1.560000e-02
1,Level k of cost of effort,1.709268e-16,8.218133e-12,1.709215e-16,1.709268e-16,1.710000e-16
2,Intrinsic motivation s,3.722260e-06,1.162147e-04,3.722242e-06,3.722260e-06,3.720000e-06


In [95]:
df['samplenw'] = (df['treatment'].isin(['1.1','1.2','1.3','3.1','3.2','4.1','4.2','10'])).astype(int)
stvale_spec = [0.003, 0.13, 1.16, 0.75, 5e-6]#alpha_init, a_init, beta_init, delta_init, gift_init

In [565]:
# Define the f(x,θ) to estimate all parameters but the probability weight in the exp case

# xdata is the vector containing the explanatory variables:

# gd is gift dummy
# dd is delay dummy
# dw is delay weeks
# paychar is pay in charity treatment
# dc is dummy charity

# parameters:

# g, k, s are the same parameters from before
# alpha is the pure altruism coefficient
# a is the warm glow coefficient
# gift is the gift exchange coefficient Δs
# beta is the present bias paramater
# delta is the (weekly) discount factor
def treatment_a(effort,k_scaler,s_scaler,type,st_values):
    def noweight(xdata, g, k, s, alpha, a, gift, beta, delta):
        pay100 = xdata[0]
        gd = xdata[1]
        dd = xdata[2]
        dw = xdata[3]
        paychar = xdata[4]
        dc = xdata[5]
        if type == "exp":
            check1 = k/k_scaler
            check2 = s/s_scaler + gift*0.4*gd + (beta**dd)*(delta**dw)*pay100 + alpha*paychar +a*0.01*dc
        elif type == "power":
            check1= max(k/k_scaler, 1e-115)
            check2= np.maximum(s/s_scaler + gift*0.4*gd + (beta**dd)*(delta**dw)*pay100 + alpha*paychar + a*0.01*dc, 1e-10)  
        f_x = (-1/g * np.log(check1) + 1/g*np.log(check2))
        return f_x

# Find the solution to the problem by non-linear least squares 
    
    st_valuesnoweight = np.concatenate((st_values,stvale_spec)) # starting values

    args = [df.loc[df['samplenw']==1].payoff_per_100, df.loc[df['samplenw']==1].gift_dummy, df.loc[df['samplenw']==1].delay_dummy,
            df.loc[df['samplenw']==1].delay_wks, df.loc[df['samplenw']==1].payoff_charity_per_100, df.loc[df['samplenw']==1].dummy_charity]

    sol = opt.curve_fit(noweight, 
                    args,
                    df.loc[df['samplenw']==1,effort],
                    st_valuesnoweight)
    
    sol_result = [0]*8
    se_result = [0]*8
    se = np.sqrt(np.diagonal(sol[1]))
    for i in range(3,len(sol[0])):
        if i==5:
            sol_result[i] = '{0:.2e}'.format(Decimal(sol[0][i]))
            se_result[i] = '{0:.2e}'.format(Decimal(se[i]))   
        else:
            sol_result[i] = round(sol[0][i],3)
            se_result[i] = round(se[i],3)
    if type == "power":
        nwest_aut = [20.51815, 5.17e-13, 3.26977, 0.0064462, 0.1818249, 0.0000204, 1.357934, 0.7494928]
        sse_our = np.sum((noweight(args,*sol[0])-df.loc[df['samplenw']==1,effort])**2)
        sse_aut = np.sum((noweight(args,*nwest_aut)-df.loc[df['samplenw']==1,effort])**2)
        #return sol[0],np.sqrt(np.diagonal(sol[1])),sse_our,sse_aut
        return sol_result,se_result,sse_our,sse_aut
    elif type == "exp":
        #return sol[0],np.sqrt(np.diagonal(sol[1]))
        return sol_result,se_result


In [527]:
for i in range(1,3):
    print(i)

1
2


In [566]:
be56,se56 = treatment_a("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",st_values = st_values_exp)
be56

C:\Users\ua896\AppData\Local\Temp\ipykernel_44588\551879273.py:33: RuntimeWarning: invalid value encountered in log
  f_x = (-1/g * np.log(check1) + 1/g*np.log(check2))


[0, 0, 0, 0.004, 0.143, '2.35e-5', 1.237, 0.754]

In [563]:
bp53,se53,sse_our,sse_aut  = treatment_a("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type = "power",st_values = st_values_power)

In [537]:
# Create and save the dataframe for table 5 NLS estimates. We leave standard errors for all parameters instead of confidence intervals for some.
# Point estimates for power case do not coincide precisely as explained above. Standard errors do not coincide precisely because of 
# the differences in the point estimates and because we leave here non-robust standard errors provided by curve_fit. To see an implementation of 
# the formula for robust standard errors please refer to the python or julia notebooks for table_1 of augenblick-rabin or table_1 of bruhin-fehr-schunk.
# The formula is the same as in the cited notebooks without considering the clustering at the individual level.

from decimal import Decimal

params_name = ["Curvature γ of cost function", "Level k of cost of effort", "Intrinsic motivation s","Social preferences α",
                "Warm glow coefficient a","Gift exchange Δs", "Present bias β","(Weekly) discount factor δ"]

#be5 = ['{0:.3}'.format(Decimal(para[0])), '{0:.2e}'.format(Decimal(para[1])), '{0:.2e}'.format(Decimal(para[2])),
       #round(be56[0][3],3), round(be56[0][4],3), '{0:.2e}'.format(Decimal(be56[0][5])), round(be56[0][6],2), round(be56[0][7],2)]
#se5 = ['{0:.3}'.format(Decimal(se_para[0])), '{0:.2e}'.format(Decimal(se_para[1])), '{0:.2e}'.format(Decimal(se_para[2])),
       #round(be56[1][3],3), round(be56[1][4],3), '{0:.2e}'.format(Decimal(be56[1][5])), round(be56[1][6],2), round(be56[1][7],2)]

#bp5 = ['{0:.5}'.format(Decimal(p_para[0])), '{0:.2e}'.format(Decimal(p_para[1])), '{0:.2e}'.format(Decimal(p_para[2])),#
       #round(bp53[0][3],4), round(bp53[0][4],4), '{0:.2e}'.format(Decimal(bp53[0][5])), round(bp53[0][6],4), round(bp53[0][7],4)]
#sp5 = ['{0:.5}'.format(Decimal(se_p_para[0])), '{0:.2e}'.format(Decimal(se_p_para[1])), '{0:.2e}'.format(Decimal(se_p_para[2])),
       #round(bp53[1][3],4), round(bp53[1][4],4), '{0:.2e}'.format(Decimal(bp53[1][5])), round(bp53[1][6],4), round(bp53[1][7],4)]

t5 = pd.DataFrame({'parameters':params_name,'power_est':bp5,'power_se':sp5,'exp_est':be5,'exp_se':se5})
print('Table 5: non-linear-least-squares estimates of behavioural parameters')
display(t5)

Table 5: non-linear-least-squares estimates of behavioural parameters


,parameters,power_est,power_se,exp_est,exp_se
0,Curvature γ of cost function,21.194,7.3990,0.0156,0.00415
1,Level k of cost of effort,5.95e-72,3.33e-70,1.71e-16,1.49e-15
2,Intrinsic motivation s,1.38e-6,4.93e-6,3.72e-6,9.16e-6
3,Social preferences α,0.0132,0.0296,0.004,0.011
4,Warm glow coefficient a,0.2649,0.2877,0.143,0.143
5,Gift exchange Δs,3.25e-5,8.14e-5,2.35e-5,4.82e-5
6,Present bias β,1.6158,2.0528,1.24,1.3
7,(Weekly) discount factor δ,0.7507,0.2916,0.75,0.24


In [564]:
print('The sum of squared errors using our estimates is: ' + str(sse_our))
print("The sum of squared errors using the authors'estimates is: " + str(sse_aut))
print('The small difference between the Stata sse and the sse computed by us are most likely due to rounding.')

The sum of squared errors using our estimates is: 1539.4456623539854
The sum of squared errors using the authors'estimates is: 1543.0565792686543
The small difference between the Stata sse and the sse computed by us are most likely due to rounding.


In [263]:
df['samplepr'] = (df['treatment'].isin(['1.1','1.2','1.3','6.1','6.2'])).astype(int)

In [299]:
prob_weight_init = [0.2]
curv_init = [0.5]

In [514]:
def treatment_prob(effort,k_scaler,s_scaler,type,st_values,curve):
    def probweight(xdata, g, k, s, p_weight):      
        pay100 = xdata[0]
        wd = xdata[1]
        prob = xdata[2]
    
        if type == "exp":
            check1 = k/k_scaler
            check2 = s/s_scaler+ p_weight**wd*prob*pay100**curve
        else :
            check1 = max(k/k_scaler, 1e-115)
            check2 = np.maximum(s/s_scaler + p_weight**wd*prob*pay100**curve, 1e-10)
        
        f_x = (-1/g * np.log(check1) + 1/g*np.log(check2))
        return f_x
    
    st_valuesprobweight = np.concatenate((st_values,prob_weight_init))
    args = [df.loc[df['samplepr']==1].payoff_per_100, df.loc[df['samplepr']==1].weight_dummy, df.loc[df['samplepr']==1].prob]
    sol = opt.curve_fit(probweight,
                    args,
                    df.loc[df['samplepr']==1,effort],
                    st_valuesprobweight)
    #Transform to easily understanding and comparable format
    solo = [i/j for i,j in zip(sol[0],[1,k_scaler,s_scaler,1,1])]
    se = [i/j for i,j in zip(np.sqrt(np.diagonal(sol[1])),[1,k_scaler,s_scaler,1,1])]
    sol_result = [0]*4
    se_result = [0]*4
    for i in range(0,len(solo)):
        if i==1 or i==2:
            sol_result[i] = '{0:.2e}'.format(Decimal(solo[i]))
            se_result[i] = '{0:.2e}'.format(Decimal(se[i]))
        else:
            sol_result[i] = round(solo[i],4)
            se_result[i] = round(se[i],4)        
    #solo = '{0:.2e}'.format(Decimal(solo))
    return np.append(sol_result,curve),np.append(se_result,0) 


In [515]:
be64,se64= treatment_prob("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",st_values = st_values_exp,curve =1)
be65,se65 = treatment_prob("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",st_values = st_values_exp,curve =0.88)
bp61,sp61 = treatment_prob("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type = "power",st_values = st_values_power,curve =1)
bp62,sp62 = treatment_prob("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type = "power",st_values = st_values_power,curve =0.88)
be64

array(['0.0134', '2.42e-14', '1.64e-5', '0.239', '1'], dtype='<U32')

In [492]:
#be64[2] = '{0:.2e}'.format(Decimal(be64[2]))
be64[0] = round(be64[0],4)
be64[2] = '{0:.2e}'.format(Decimal(be64[2]))
be64[3]

0.239

In [504]:
cc = [0]*5
for i in range (0,(len(be64))):
    if i ==1 or i==2:
        cc[i] = '{0:.2e}'.format(Decimal(be64[i]))
        #print(i)
    else:
        cc[i] = round(be64[i],4)
    #print(be64[i])
cc

[0.0134, '2.42e-14', '1.64e-5', 0.239, 1.0]

In [522]:
def treatment_prob_curve(effort,k_scaler,s_scaler,type,st_values):
    def probweight(xdata, g, k, s, p_weight,curve):      
        pay100 = xdata[0]
        wd = xdata[1]
        prob = xdata[2]
    
        if type == "exp":
            check1 = k/k_scaler
            check2 = s/s_scaler+ p_weight**wd*prob*pay100**curve
        else :
            check1 = max(k/k_scaler, 1e-115)
            check2 = np.maximum(s/s_scaler + p_weight**wd*prob*pay100**curve, 1e-10)
        
        f_x = (-1/g * np.log(check1) + 1/g*np.log(check2))
        return f_x
    
    st_valuesprobweight = np.concatenate((st_values,prob_weight_init,curv_init))
    #st_valuesprobweight = np.concatenate((st_valuesprobweight,curv_init))
    #args = [df.loc[df['samplepr']==1].payoff_per_100, df.loc[df['samplepr']==1].weight_dummy, df.loc[df['samplepr']==1].prob]
    sol = opt.curve_fit(probweight,
                    args,
                    df.loc[df['samplepr']==1,effort],
                    st_valuesprobweight)
    solo = [i/j for i,j in zip(sol[0],[1,k_scaler,s_scaler,1,1])]
    se = [i/j for i,j in zip(np.sqrt(np.diagonal(sol[1])),[1,k_scaler,s_scaler,1,1])]
    sol_result = [0]*5
    se_result = [0]*5
    for i in range(0,len(solo)):
        if i==1 or i==2:
            sol_result[i] = '{0:.2e}'.format(Decimal(solo[i]))
            se_result[i] = '{0:.2e}'.format(Decimal(se[i]))
        else:
            sol_result[i] = round(solo[i],4)
            se_result[i] = round(se[i],4)   
    return sol_result,se_result

In [523]:
be66, se66 = treatment_prob_curve("buttonpresses_nearest_100",k_scaler = 1e+16, s_scaler= 1e+6,type = "exp",st_values = st_values_exp)
bp63,sp63 = treatment_prob_curve("logbuttonpresses_nearest_100",k_scaler = 1e+57, s_scaler= 1e+6,type = "power",st_values = st_values_power)

In [525]:
pnames = ["Curvature γ of cost function", "Level k of cost of effort", "Intrinsic motivation s", "Probability weighting π (1%) (in %)",
          "Curvature of utility over piece rate"]

t6 = pd.DataFrame({'parameters':pnames,'p_est1':bp61,'p_se1':sp61,'p_est2':bp62,'p_se2':sp62,
                   'p_est3':bp63,'p_se3':sp63,
                   'e_est4':be64,'e_se4':se64,'e_est5':be65,'e_se5':se65,'e_est6':be66,
                   'e_se6':se66})
display(t6)

,parameters,p_est1,p_se1,p_est2,p_se2,p_est3,p_se3,e_est4,e_se4,e_est5,e_se5,e_est6,e_se6
0,Curvature γ of cost function,20.9475,5.7752,18.9623,5.2711,19.6386,17.3201,0.0134,0.0026,0.0119,0.0023,0.0072,0.0029
1,Level k of cost of effort,3.89e-71,1.70e-69,1.96e-64,7.68e-63,1.01e-66,1.36e-64,2.42e-14,1.29e-13,7.50e-13,3.56e-12,5.46e-8,3.70e-7
2,Intrinsic motivation s,1.57e-6,4.23e-6,5.96e-6,1.47e-5,3.75e-6,4.17e-5,1.64e-5,2.40e-5,5.55e-5,7.20e-5,3.14e-3,7.50e-3
3,Probability weighting π (1%) (in %),0.1928,0.173,0.3732,0.3042,0.2952,1.5657,0.239,0.1427,0.466,0.2472,4.2961,5.4624
4,Curvature of utility over piece rate,1,0,0.88,0,0.9235,0.9289,1,0,0.88,0,0.4679,0.2368


In [32]:
TT = list(range(-5,5,1))
df_try = pd.DataFrame(TT,columns=["seq"])
df_try['seq'].values.min()
from itertools import repeat
def ttt(p):
    if df_try.seq.values.min() <= p:
        return np.zeros(len(df_try.seq))
    f_x = df_try.seq*2
    return f_x
       
#if k <= 0 or (s+pay100).values.min() <= 0:
        #return 1e-15